<a href="https://colab.research.google.com/github/EskelinenElias/ADAML-project1/blob/main/data_modeling_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADAML Project Work: Intermediary Submission 2 - Data Modeling Plan

LUT University \
BM20A6100 Advanced Data Analysis and Machine Learning - Blended teaching, Lpr 1.9.2025-12.12.2025 \
1st Period - Project work - Intermediary Submission 2 - Data modeling plan

Elias Eskelinen, Vili Niemelä & Matti Aalto \
25.9.2025

---

## Intermediary Submission 2 - Instructions

Based on your initial evaluation, the received feedback and the strategies of other groups, perform data pretreatment. There will be a max. 2 pages addition to your project PDF for the pretreatment part.

Breakdown of the points:

0.5p   - feedback incorporation and adjustment of the initial pretreatment plan
0.5p  - data centering and scaling techniques
0.5p  - evaluation of extreme/ missing values and mitigating actions, data synchronization (for time series), sampling
0.5p   - visualizing pretreated data
Modelling plan

Make a modelling plan for your modelling goal. Create a flowchart of the operations needed to accomplish the task, and assign roles to the team members for each step. The start of each flow chart should be the cleaned data, and the end of it should be the modelling goal achieved. If there is more than one modelling goal, the flowchart can take multiple paths. There will be a max. 2 pages addition to your project PDF for the modelling plan part.

0.5p     - modelling goal (What is the purpose of your data analysis?);
0.5p     - model calibration strategy: tools used, methodology, data used;
0.5p     - model validation strategy: scope of validation, methodology;
0.5p     - model testing strategy: metrics and data used;
0.5p     - description of the mathematical methods used in modelling;
0.5p     - model diagram/ operations flowchart.


## Data onboarding

Fetch data from kaggle:

In [25]:
import kagglehub
import os

# Fetch the latest version of the dataset from kaggle
data_dir = kagglehub.dataset_download("edumagalhaes/quality-prediction-in-a-mining-process")

data_path = os.path.join(data_dir, os.listdir(data_dir)[0])
print("Path to dataset files:", data_path)

Using Colab cache for faster access to the 'quality-prediction-in-a-mining-process' dataset.
Path to dataset files: /kaggle/input/quality-prediction-in-a-mining-process/MiningProcess_Flotation_Plant_Database.csv


Loading the data

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv(data_path)

Formatting the data

In [27]:
# Format date column as dates
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

# Format other columns as decimal numbers
for c in data.columns[1:]: data[c] = data[c].str.replace(',', '.').astype(float)

Resampling the data

In [22]:
# Resample the data to 1 hour frequency
data = data.groupby("date").mean(numeric_only=True).reset_index()

Check data size and shape

In [24]:
print(data.shape)

(4097, 24)


## Data preprocessing

Normalizing the data

In [34]:
normalized_cols = data[data.columns[1:]] # Normalize all columns except the date column

means = normalized_cols.mean()
stds = normalized_cols.std()

data[data.columns[1:]] = (normalized_cols - means) / stds

print(data.mean())
print(data.std())

date                            2017-06-16 03:27:22.656549376
% Iron Feed                                              -0.0
% Silica Feed                                            -0.0
Starch Flow                                               0.0
Amina Flow                                                0.0
Ore Pulp Flow                                            -0.0
Ore Pulp pH                                              -0.0
Ore Pulp Density                                          0.0
Flotation Column 01 Air Flow                              0.0
Flotation Column 02 Air Flow                              0.0
Flotation Column 03 Air Flow                             -0.0
Flotation Column 04 Air Flow                              0.0
Flotation Column 05 Air Flow                             -0.0
Flotation Column 06 Air Flow                              0.0
Flotation Column 07 Air Flow                              0.0
Flotation Column 01 Level                                 0.0
Flotatio